In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def fit_gaussian(x_points, y_points):
    """
    Fit a Gaussian distribution to a set of points.
    
    Parameters:
    -----------
    x_points : array-like
        X coordinates (independent variable)
    y_points : array-like
        Y coordinates (density values at corresponding x points)
    
    Returns:
    --------
    params : tuple
        (mean, std_dev, amplitude) of the fitted Gaussian
    gaussian_func : function
        The fitted Gaussian function that can be called with new x values
    """
    
    def gaussian(x, mean, std_dev, amplitude):
        """Gaussian function"""
        return amplitude * np.exp(-((x - mean) ** 2) / (2 * std_dev ** 2))
    
    # Convert to numpy arrays
    x_points = np.array(x_points)
    y_points = np.array(y_points)
    
    # Initial guess for parameters
    mean_guess = np.sum(x_points * y_points) / np.sum(y_points)
    amplitude_guess = np.max(y_points)
    std_dev_guess = np.sqrt(np.sum(y_points * (x_points - mean_guess)**2) / np.sum(y_points))
    
    initial_guess = [mean_guess, std_dev_guess, amplitude_guess]
    
    # Fit the Gaussian
    params, covariance = curve_fit(gaussian, x_points, y_points, p0=initial_guess)
    
    mean, std_dev, amplitude = params
    
    # Create a function for the fitted Gaussian
    def fitted_gaussian(x):
        return gaussian(x, mean, std_dev, amplitude)
    
    return params, fitted_gaussian


# Example usage:
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    
    # Sample data points (x, density values)
    x_data = np.array([0, 1, 2, 3, 4, 5, 6])
    y_data = np.array([5, 15, 35, 40, 35, 15, 5])
    
    # Fit the Gaussian
    params, fitted_func = fit_gaussian(x_data, y_data)
    mean, std_dev, amplitude = params
    
    print(f"Fitted parameters:")
    print(f"Mean: {mean:.3f}")
    print(f"Standard deviation: {std_dev:.3f}")
    print(f"Amplitude: {amplitude:.3f}")
    
    # Plot the results
    x_smooth = np.linspace(x_data.min() - 1, x_data.max() + 1, 200)
    y_fitted = fitted_func(x_smooth)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x_data, y_data, color='red', s=100, label='Data points', zorder=3)
    plt.plot(x_smooth, y_fitted, 'b-', linewidth=2, label='Fitted Gaussian')
    plt.xlabel('x')
    plt.ylabel('Density')
    plt.title('Gaussian Distribution Fit')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import nibabel as nib

def fit_gaussian(x_points, y_points):
    """Fit a Gaussian distribution to a set of points."""
    
    def gaussian(x, mean, std_dev, amplitude):
        return amplitude * np.exp(-((x - mean) ** 2) / (2 * std_dev ** 2))
    
    x_points = np.array(x_points)
    y_points = np.array(y_points)
    
    # Initial guess
    mean_guess = np.sum(x_points * y_points) / np.sum(y_points)
    amplitude_guess = np.max(y_points)
    std_dev_guess = np.sqrt(np.sum(y_points * (x_points - mean_guess)**2) / np.sum(y_points))
    
    initial_guess = [mean_guess, std_dev_guess, amplitude_guess]
    
    params, covariance = curve_fit(gaussian, x_points, y_points, p0=initial_guess, maxfev=5000)
    
    mean, std_dev, amplitude = params
    
    def fitted_gaussian(x):
        return gaussian(x, mean, std_dev, amplitude)
    
    return params, fitted_gaussian

def compute_histogram(ndarray):
    flat_array = ndarray.flatten()
    bins = np.arange(-4000, 4001, 10)
    histogram_values, bin_edges = np.histogram(flat_array, bins=bins)
    histogram_list = histogram_values.tolist()
    return histogram_list, bin_edges

def get_points_around_peak(histogram, bin_edges, peak_index, num_points=3):
    start_index = max(0, peak_index - num_points)
    end_index = min(len(histogram), peak_index + num_points + 1)
    
    x_points = bin_edges[start_index:end_index]
    y_points = histogram[start_index:end_index]
    
    return x_points, y_points

def image_normalize(file_path, n_sample_points=10):

    # Load image
    nii_img = nib.load(file_path)
    data = nii_img.get_fdata()
    data_noBG = data[data > -249]

    # Normalize image
    histogram, bin_edges = compute_histogram(data_noBG)

    try:
        voxels = data_noBG.flatten()
        voxels = voxels[np.isfinite(voxels)]
        overall_mean = np.mean(voxels)
        mid_point = len(bin_edges[bin_edges <= overall_mean])
        lower_half = histogram[:mid_point]

        peak_x = np.argmax(lower_half)
        x, y = get_points_around_peak(histogram, bin_edges, peak_x, num_points=n_sample_points)

        params, fitted_func = fit_gaussian(x, y)
        mean, std_dev, amplitude = params
    except Exception as e:
        print(f"Error in fitting Gaussian: {e}")
        peak_x = np.argmax(histogram)
        x, y = get_points_around_peak(histogram, bin_edges, peak_x, num_points=n_sample_points)
        params, fitted_func = fit_gaussian(x, y)
        mean, std_dev, amplitude = params

    normalized_data = (data - mean) / std_dev

    # Create new image
    new_img = nib.Nifti1Image(normalized_data.astype(np.float32), nii_img.affine, nii_img.header)

    return new_img

In [ ]:
from pathlib import Path

input_folder = Path(r"D:\Kananat\Data\alldata_new")
result_folder = Path(r"D:\Kananat\Data\alldata_new_normalized")

i = 0
for file_path in input_folder.glob("*.nii.gz"):
    print(f"{i}: Processing {file_path.name}")
    normalized_img = image_normalize(file_path, n_sample_points=5)
    output_path = result_folder / file_path.name
    nib.save(normalized_img, output_path)
    i += 1

In [45]:
from pathlib import Path

input_folder = Path(r"D:\Kananat\Data\alldata_new")
result_folder = Path(r"D:\Kananat\Data\alldata_new_normalized")

def image_normalize(file_path):

    # Load image
    nii_img = nib.load(file_path)
    data = nii_img.get_fdata()
    data = np.clip(data, -250, 2000)
    normalized_data = (data+250)/2250

    new_img = nib.Nifti1Image(normalized_data.astype(np.float32), nii_img.affine, nii_img.header)

    return new_img

i = 0
for file_path in input_folder.glob("*.nii.gz"):
    print(f"{i}: Processing {file_path.name}")
    normalized_img = image_normalize(file_path)
    output_path = result_folder / file_path.name
    nib.save(normalized_img, output_path)
    i += 1

0: Processing 47-16872 L.nii.gz
1: Processing 47-16872 R.nii.gz
2: Processing 47-22136 L.nii.gz
3: Processing 47-22136 R.nii.gz
4: Processing 47-4881 L 2014.nii.gz
5: Processing 47-4881 L 2018.nii.gz
6: Processing 47-4881 R 2014.nii.gz
7: Processing 47-4881 R 2018.nii.gz
8: Processing 48-26453 L.nii.gz
9: Processing 48-5955 L.nii.gz
10: Processing 48-5955 R.nii.gz
11: Processing 49-18165 L.nii.gz
12: Processing 49-18165 R.nii.gz
13: Processing 49-3614 L.nii.gz
14: Processing 49-3614 R.nii.gz
15: Processing 50-11620 L 20250213.nii.gz
16: Processing 50-11620 R 20250213.nii.gz
17: Processing 50-30909 R.nii.gz
18: Processing 51-26987 L.nii.gz
19: Processing 51-26987 R.nii.gz
20: Processing 51-28114 L.nii.gz
21: Processing 51-28114 R.nii.gz
22: Processing 51-3282 L.nii.gz
23: Processing 51-3282 R.nii.gz
24: Processing 52-11116 L.nii.gz
25: Processing 52-11116 R.nii.gz
26: Processing 52-15073 L.nii.gz
27: Processing 52-15073 R.nii.gz
28: Processing 52-15242 L.nii.gz
29: Processing 52-15242 R